[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/pytorch-practice/blob/master/_jupyternotebooks/ngrok_test_env.ipynb)

# [ngrok](https://ngrok.com/) 세팅

In [ ]:
NGROK_TOKEN = "hello_hommy"
PASSWORD = "hello_hommy"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh
launch_ssh(NGROK_TOKEN, PASSWORD)

# 테스트 코드

## 환경 설정

In [ ]:
!git clone https://github.com/shhommychon/pytorch-imgrecog.git

In [ ]:
import sys
sys.path.append("/content/pytorch-imgrecog")
import image_recognition as myir

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

print (f"PyTorch version: {torch.__version__}")

In [ ]:
print(f"cuda available? : {torch.cuda.is_available()}")
_device_count = torch.cuda.device_count()
print(f"cuda device count? : {_device_count}")
print()

device = torch.device(f"cuda:{_device_count-1}" if torch.cuda.is_available() else "cpu")
print (f"device: {device}")

## 샘플 데이터 (ImageNet)

### dataset

In [ ]:
from torchvision import datasets, transforms

In [ ]:
fashion_train = datasets.FashionMNIST(
    root="/content/data/", train=True, download=True, 
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [ ]:
fashion_train

In [ ]:
fashion_train, fashion_val = torch.utils.data.random_split(fashion_train, [50000, 10000])

In [ ]:
fashion_train

In [ ]:
fashion_val

In [ ]:
fashion_test = datasets.FashionMNIST(
    root="/content/data/", train=False, download=True, 
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [ ]:
fashion_test

In [ ]:
fashion_test.data.shape

### data iterator

In [ ]:
BATCH_SIZE = 64
train_iter = torch.utils.data.DataLoader(fashion_train, batch_size=BATCH_SIZE, shuffle=True)
val_iter = torch.utils.data.DataLoader(fashion_val, batch_size=BATCH_SIZE, shuffle=True)
test_iter = torch.utils.data.DataLoader(fashion_test, batch_size=BATCH_SIZE, shuffle=False)

## 모델

In [ ]:
learning_rate = 1e-3

### torch official implementation

In [ ]:
# torch_model
# T = pass
# T_loss = nn.CrossEntropyLoss()
# T_optim = optim.Adam(T.parameters(),lr=learning_rate)

In [ ]:
# print(T)

### my implementation

In [ ]:
# my_model
# M = pass
# M_loss = nn.CrossEntropyLoss()
# M_optim = optim.Adam(M.parameters(),lr=learning_rate)

In [ ]:
# print(M)

## 학습

In [ ]:
from tqdm import tqdm

def func_eval(model, data_iter, device):
    with torch.no_grad():
        n_total, n_correct = 0, 0

        model.eval()
        for batch_in, batch_out in tqdm(data_iter):
            y_true = batch_out.to(device)
            model_pred = model(batch_in.view(-1, 1, 28, 28).to(device))
            _, y_pred = torch.max(model_pred.data,1)

            n_correct += (y_pred==y_true).sum().item()
            n_total += batch_in.size(0)

        eval_accr = n_correct / n_total

        model.train()

    return eval_accr

EPOCHS = 10

### torch official implementation

In [ ]:
print ("start training...")
T.init_param()
T.train()

T_dict = list()
# T_best_val_acc = 0.0

for e_idx in range(EPOCHS):
    e_idx += 1
    print(f"epoch #{e_idx:0>2}")

    print(f"\t{'training': >16}... ", end='')
    loss_val_sum = 0.0
    for batch_in, batch_out in tqdm(train_iter):
        # forward
        y_pred = T.forward(batch_in.view(-1, 1, 28, 28).to(device))
        loss_out = T_loss(y_pred, batch_out.to_device)

        # update
        T_optim.zero_grad()
        loss_out.backward() # Backpropagate
        T_optim.step()
        loss_val_sum += loss_out
    loss_val_avg = loss_val_sum/len(train_iter)

    print(f"\t{'evaluating train': >16}... ", end='')
    train_accr = func_eval(T, train_iter, device)
    print(f"\t{'evaluating val': >16}... ", end='')
    val_accr = func_eval(T, val_iter, device)

    print(f"epoch #{e_idx:0>2}: "
        + f"loss = [{loss_val_avg:.4f}] / "
        + f"train_accr = [{train_accr:.4f}] / "
        + f"val_accr = [{val_accr:.4f}]\n"
    )
    T_dict.append({
        "loss": loss_val_avg,
        "train_accr": train_accr,
        "val_accr": val_accr
    })
    
    # if T_best_val_acc < val_accr:
    #     T_best_val_acc = val_accr
    #     torch.save(T, f"best-model-torch.pt") 

del e_idx, loss_val_avg, train_accr, val_accr
print("finished training!")

### my implementation

In [ ]:
print ("start training...")
M.init_param()
M.train()

M_dict = list()
# M_best_val_acc = 0.0

for e_idx in range(EPOCHS):
    e_idx += 1
    print(f"epoch #{e_idx:0>2}")

    print(f"\t{'training': >16}... ", end='')
    loss_val_sum = 0.0
    for batch_in, batch_out in tqdm(train_iter):
        # forward
        y_pred = M.forward(batch_in.view(-1, 1, 28, 28).to(device))
        loss_out = M_loss(y_pred, batch_out.to_device)

        # update
        M_optim.zero_grad()
        loss_out.backward() # Backpropagate
        M_optim.step()
        loss_val_sum += loss_out
    loss_val_avg = loss_val_sum/len(train_iter)

    print(f"\t{'evaluating train': >16}... ", end='')
    train_accr = func_eval(M, train_iter, device)
    print(f"\t{'evaluating val': >16}... ", end='')
    val_accr = func_eval(M, val_iter, device)

    print(f"epoch #{e_idx:0>2}: "
        + f"loss = [{loss_val_avg:.4f}] / "
        + f"train_accr = [{train_accr:.4f}] / "
        + f"val_accr = [{val_accr:.4f}]\n"
    )
    M_dict.append({
        "epoch": e_idx,
        "loss": loss_val_avg,
        "train_accr": train_accr,
        "val_accr": val_accr
    })
    
    # if M_best_val_acc < val_accr:
    #     M_best_val_acc = val_accr
    #     torch.save(M, f"best-model-my.pt") 

del e_idx, loss_val_avg, train_accr, val_accr
print("finished training!")

## 평가

In [ ]:
print("evaluating test... ", end='')
T_test_accr = func_eval(T, test_iter, device)

In [ ]:
print("evaluating test... ", end='')
M_test_accr = func_eval(M, test_iter, device)

In [ ]:
T_test_accr, M_test_accr

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
T_df = pd.DataFrame(T_dict)
M_df = pd.DataFrame(M_dict)

fig = plt.figure(figsize=(12, 12))

ax1 = fig.add_subplot(221)
ax1.plot(T_df["epoch"], T_df["loss"], "c-")
ax1.set_ylabel("loss")
ax1.set_xlabel("epoch")
ax1.set_title("torch implementation loss")

ax2 = fig.add_subplot(222)
ax2.plot(M_df["epoch"], M_df["loss"], "c-")
ax2.set_ylabel("loss")
ax2.set_xlabel("epoch")
ax2.set_title("my implementation loss")

ax3 = fig.add_subplot(223)
ax3.plot(T_df["epoch"], T_df["train_accr"], "g:", label="train")
ax3.plot(T_df["epoch"], T_df["val_accr"], "y--", label="val")
ax3.set_ylabel("accuracy")
ax3.set_xlabel("epoch")
ax3.set_title("torch implementation accuracy")
ax3.legend()

ax4 = fig.add_subplot(224)
ax4.plot(M_df["epoch"], M_df["train_accr"], "g:", label="train")
ax4.plot(M_df["epoch"], M_df["val_accr"], "y--", label="val")
ax4.set_ylabel("accuracy")
ax4.set_xlabel("epoch")
ax4.set_title("my implementation accuracy")
ax4.legend()

plt.show()